<img src="agri_image2.png" alt="Sample Image" width="900" height="200">

## Table of Contents

<a href=#BC> Background Context</a>

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Data Collection and Description</a>

<a href=#three>3. Loading Data </a>

<a href=#four>4. Data Cleaning and Filtering</a>

<a href=#five>5. Exploratory Data Analysis (EDA)</a>

<a href=#six>6. Modeling </a>

<a href=#seven>7. Evaluation and Validation</a>

<a href=#eight>8. Final Model</a>

<a href=#nine>9. Conclusion and Future Work</a>

---
 <a id="BC"></a>
## **Background Context**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Introduce the project, outline its goals, and explain its significance.
* **Details:** Include information about the problem domain, the specific questions or challenges the project aims to address, and any relevant background information that sets the stage for the work.
---

<a href=#one></a>
## **Importing Packages**
<a href=#cont>Back to Table of Contents</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

---
<a href=#three></a>
## **Loading Data**
<a href=#cont>Back to Table of Contents</a>

In [2]:
df = pd.read_csv("co2_emissions_from_agri.csv")

---
<a href=#four></a>
## **Data Cleaning and Filtering**
<a href=#cont>Back to Table of Contents</a>

---
<a href=#five></a>
## **Exploratory Data Analysis (EDA)**
<a href=#cont>Back to Table of Contents</a>

---
<a href=#six></a>
## **Modeling**
<a href=#cont>Back to Table of Contents</a>

In [9]:
# Feature Selection: Select relevant features for modeling
features = numeric_df.drop(columns=['total_emission'])
target = numeric_df['total_emission']

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [11]:
# Multiple Linear Regression
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred_linear = linear_regressor.predict(X_test)

In [12]:
# Decision Tree Regression
tree_regressor = DecisionTreeRegressor(random_state=42)
tree_regressor.fit(X_train, y_train)
y_pred_tree = tree_regressor.predict(X_test)

In [13]:
# Random Forest Regression
forest_regressor = RandomForestRegressor(random_state=42)
forest_regressor.fit(X_train, y_train)
y_pred_forest = forest_regressor.predict(X_test)
mse_forest = mean_squared_error(y_test, y_pred_forest)
r2_forest = r2_score(y_test, y_pred_forest)

In [18]:
# Feature importances
tree_importances = tree_regressor.feature_importances_
forest_importances = forest_regressor.feature_importances_

# Plot feature importances
features = x.columns
indices = np.argsort(forest_importances)[::-1]

plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.title("Feature Importances - Random Forest")
plt.bar(range(X.shape[1]), forest_importances[indices], align="center")
plt.xticks(range(X.shape[1]), features[indices], rotation=90)

plt.subplot(2, 1, 2)
plt.title("Feature Importances - Decision Tree")
plt.bar(range(X.shape[1]), tree_importances[indices], align="center")
plt.xticks(range(X.shape[1]), features[indices], rotation=90)

plt.tight_layout()
plt.show()

NameError: name 'x' is not defined

In [19]:
# Evaluate and compare model performance
def evaluate_model(y_test, y_pred, model_name):
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} - Mean Squared Error: {mse}, R^2 Score: {r2}")

In [20]:
# Print model evaluation results
evaluate_model(y_test, y_pred_linear, "Multiple Linear Regression")
evaluate_model(y_test, y_pred_tree, "Decision Tree Regression")
evaluate_model(y_test, y_pred_forest, "Random Forest Regression")

Multiple Linear Regression - Mean Squared Error: 5.3871848317771426e-17, R^2 Score: 1.0
Decision Tree Regression - Mean Squared Error: 1141577675.9970527, R^2 Score: 0.9834969060266138
Random Forest Regression - Mean Squared Error: 626956432.239349, R^2 Score: 0.9909364722734018


---
<a href=#seven></a>
## **Evaluation and Validation**
<a href=#cont>Back to Table of Contents</a>

---
<a href=#eight></a>
## **Final Model**
<a href=#cont>Back to Table of Contents</a>

---
<a href=#nine></a>
## **Conclusion**
<a href=#cont>Back to Table of Contents</a>